# Lesson 4-17 CNN
# Output depth
k_output = 64

# Image property
image_width = 10
image_height = 10
color_channels = 3

# Convolution filter
filter_size_width = 5
filter_size_hegiht = 5

# Input/Output
input = tf.placeholder(
    tf.float32,
    shape=[None, image_height, image_width, color_channels])

# Weight and bias
weight = tf.Variable(tf.trucated_normal(
    [filter_size_height, filter_size_width, color_channels, k_output])) # random var
bias = tf.Variable(tf.zeros(k_output))

# Apply function
conv_layer = tf.nn.conv2d(input, weight, stride=[1,2,2,1], padding='NAME')
# Add bias
conv_layer = tf.nn.bias_add(conv_layer, bias)
# Apply Activation Function
conv_layer = tf.nn.relu(conv_layer)

# 4-19 TensorFlow Max Pooling
conv_layer = tf.nn.max_pool(
    conv_layer,
    ksize=[1,2,2,1],
    strides=[1,2,2,1], # bash,height,width,channel
    padding='SAME')


In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".",one_hot=True, reshape=False)

Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz


In [5]:
import tensorflow as tf

# Parameters
learning_rate = 0.00001
epochs = 10
batch_size = 128

# Number of samples to calculate validation and accuracy
# Decrease this if you're running out of memory to calculate accuracy
test_valid_size = 256

# Network Parameters
n_classes = 10  # MNIST total classes (0-9 digits)
dropout = 0.75  # Dropout, probability to keep units


In [6]:


# Store layers weight & bias
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])), # (height, width, input_depth, output_depth)
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))}



In [7]:

def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')


In [8]:

def conv_net(x, weights, biases, dropout):
    # Layer 1 - 28*28*1 to 14*14*32
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)

    # Layer 2 - 14*14*32 to 7*7*64
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer - 7*7*64 to 1024
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output Layer - class prediction - 1024 to 10
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out


In [10]:

# tf Graph input
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

# Model
logits = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})

            # Calculate batch loss and accuracy
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.})
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                keep_prob: 1.})

            print('Epoch {:>2}, Batch {:>3} - Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                epoch + 1,
                batch + 1,
                loss,
                valid_acc))

    # Calculate Test Accuracy
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))

Epoch  1, Batch   1 - Loss: 75676.3125 Validation Accuracy: 0.121094
Epoch  1, Batch   2 - Loss: 60466.2422 Validation Accuracy: 0.097656
Epoch  1, Batch   3 - Loss: 51945.7422 Validation Accuracy: 0.074219
Epoch  1, Batch   4 - Loss: 44457.6875 Validation Accuracy: 0.085938
Epoch  1, Batch   5 - Loss: 38951.6211 Validation Accuracy: 0.082031
Epoch  1, Batch   6 - Loss: 38376.8984 Validation Accuracy: 0.101562
Epoch  1, Batch   7 - Loss: 32030.4102 Validation Accuracy: 0.097656
Epoch  1, Batch   8 - Loss: 32221.4648 Validation Accuracy: 0.117188
Epoch  1, Batch   9 - Loss: 31298.2031 Validation Accuracy: 0.136719
Epoch  1, Batch  10 - Loss: 27959.5117 Validation Accuracy: 0.152344
Epoch  1, Batch  11 - Loss: 28721.6348 Validation Accuracy: 0.160156
Epoch  1, Batch  12 - Loss: 26173.5703 Validation Accuracy: 0.156250
Epoch  1, Batch  13 - Loss: 21192.0469 Validation Accuracy: 0.144531
Epoch  1, Batch  14 - Loss: 26430.1738 Validation Accuracy: 0.179688
Epoch  1, Batch  15 - Loss: 25531.

Epoch  1, Batch 120 - Loss:  3285.0327 Validation Accuracy: 0.589844
Epoch  1, Batch 121 - Loss:  4072.5291 Validation Accuracy: 0.582031
Epoch  1, Batch 122 - Loss:  4443.0444 Validation Accuracy: 0.585938
Epoch  1, Batch 123 - Loss:  4628.3018 Validation Accuracy: 0.582031
Epoch  1, Batch 124 - Loss:  4632.0742 Validation Accuracy: 0.613281
Epoch  1, Batch 125 - Loss:  4336.3218 Validation Accuracy: 0.601562
Epoch  1, Batch 126 - Loss:  5195.1294 Validation Accuracy: 0.578125
Epoch  1, Batch 127 - Loss:  3422.4980 Validation Accuracy: 0.593750
Epoch  1, Batch 128 - Loss:  4313.1997 Validation Accuracy: 0.597656
Epoch  1, Batch 129 - Loss:  5453.3062 Validation Accuracy: 0.593750
Epoch  1, Batch 130 - Loss:  4720.3306 Validation Accuracy: 0.601562
Epoch  1, Batch 131 - Loss:  4454.7822 Validation Accuracy: 0.601562
Epoch  1, Batch 132 - Loss:  2941.3767 Validation Accuracy: 0.609375
Epoch  1, Batch 133 - Loss:  3398.0674 Validation Accuracy: 0.605469
Epoch  1, Batch 134 - Loss:  4354.

Epoch  1, Batch 239 - Loss:  2678.1294 Validation Accuracy: 0.718750
Epoch  1, Batch 240 - Loss:  2887.4658 Validation Accuracy: 0.699219
Epoch  1, Batch 241 - Loss:  3471.3633 Validation Accuracy: 0.710938
Epoch  1, Batch 242 - Loss:  3405.9343 Validation Accuracy: 0.707031
Epoch  1, Batch 243 - Loss:  2441.7341 Validation Accuracy: 0.710938
Epoch  1, Batch 244 - Loss:  2483.0010 Validation Accuracy: 0.714844
Epoch  1, Batch 245 - Loss:  3222.6409 Validation Accuracy: 0.707031
Epoch  1, Batch 246 - Loss:  2604.1143 Validation Accuracy: 0.703125
Epoch  1, Batch 247 - Loss:  2313.2158 Validation Accuracy: 0.707031
Epoch  1, Batch 248 - Loss:  2625.7778 Validation Accuracy: 0.699219
Epoch  1, Batch 249 - Loss:  1700.6450 Validation Accuracy: 0.699219
Epoch  1, Batch 250 - Loss:  2393.1250 Validation Accuracy: 0.703125
Epoch  1, Batch 251 - Loss:  3079.1040 Validation Accuracy: 0.710938
Epoch  1, Batch 252 - Loss:  2993.9062 Validation Accuracy: 0.714844
Epoch  1, Batch 253 - Loss:  1342.

Epoch  1, Batch 358 - Loss:  2063.0146 Validation Accuracy: 0.746094
Epoch  1, Batch 359 - Loss:  2248.5530 Validation Accuracy: 0.746094
Epoch  1, Batch 360 - Loss:  1275.7041 Validation Accuracy: 0.750000
Epoch  1, Batch 361 - Loss:  1765.5009 Validation Accuracy: 0.746094
Epoch  1, Batch 362 - Loss:  1748.4136 Validation Accuracy: 0.742188
Epoch  1, Batch 363 - Loss:  1622.5117 Validation Accuracy: 0.746094
Epoch  1, Batch 364 - Loss:  1641.1663 Validation Accuracy: 0.750000
Epoch  1, Batch 365 - Loss:  2171.8584 Validation Accuracy: 0.742188
Epoch  1, Batch 366 - Loss:  1738.4304 Validation Accuracy: 0.746094
Epoch  1, Batch 367 - Loss:  1912.2598 Validation Accuracy: 0.750000
Epoch  1, Batch 368 - Loss:  1547.2092 Validation Accuracy: 0.750000
Epoch  1, Batch 369 - Loss:  1885.1196 Validation Accuracy: 0.750000
Epoch  1, Batch 370 - Loss:  2330.3906 Validation Accuracy: 0.753906
Epoch  1, Batch 371 - Loss:  2192.6943 Validation Accuracy: 0.750000
Epoch  1, Batch 372 - Loss:   973.

Epoch  2, Batch  48 - Loss:  1694.6729 Validation Accuracy: 0.777344
Epoch  2, Batch  49 - Loss:  1608.4968 Validation Accuracy: 0.773438
Epoch  2, Batch  50 - Loss:  1030.9554 Validation Accuracy: 0.773438
Epoch  2, Batch  51 - Loss:  1832.8301 Validation Accuracy: 0.769531
Epoch  2, Batch  52 - Loss:  1609.0498 Validation Accuracy: 0.769531
Epoch  2, Batch  53 - Loss:  1347.6582 Validation Accuracy: 0.781250
Epoch  2, Batch  54 - Loss:  1434.3706 Validation Accuracy: 0.781250
Epoch  2, Batch  55 - Loss:  1475.6879 Validation Accuracy: 0.781250
Epoch  2, Batch  56 - Loss:  1436.7004 Validation Accuracy: 0.773438
Epoch  2, Batch  57 - Loss:  1986.2446 Validation Accuracy: 0.773438
Epoch  2, Batch  58 - Loss:  1624.4652 Validation Accuracy: 0.773438
Epoch  2, Batch  59 - Loss:  1445.4165 Validation Accuracy: 0.769531
Epoch  2, Batch  60 - Loss:  1181.0129 Validation Accuracy: 0.773438
Epoch  2, Batch  61 - Loss:  1569.4995 Validation Accuracy: 0.769531
Epoch  2, Batch  62 - Loss:  1410.

...

Epoch  4, Batch 380 - Loss:   855.3611 Validation Accuracy: 0.816406
Epoch  4, Batch 381 - Loss:   697.9866 Validation Accuracy: 0.816406
Epoch  4, Batch 382 - Loss:   532.3679 Validation Accuracy: 0.812500
Epoch  4, Batch 383 - Loss:   423.8725 Validation Accuracy: 0.808594
Epoch  4, Batch 384 - Loss:   418.0859 Validation Accuracy: 0.812500
Epoch  4, Batch 385 - Loss:   514.4579 Validation Accuracy: 0.816406
Epoch  4, Batch 386 - Loss:   664.5781 Validation Accuracy: 0.816406
Epoch  4, Batch 387 - Loss:   413.4270 Validation Accuracy: 0.820312
Epoch  4, Batch 388 - Loss:   441.6935 Validation Accuracy: 0.824219
Epoch  4, Batch 389 - Loss:   448.1479 Validation Accuracy: 0.820312
Epoch  4, Batch 390 - Loss:   531.4705 Validation Accuracy: 0.824219
Epoch  4, Batch 391 - Loss:   634.8822 Validation Accuracy: 0.835938
Epoch  4, Batch 392 - Loss:   529.5743 Validation Accuracy: 0.828125
Epoch  4, Batch 393 - Loss:   529.4004 Validation Accuracy: 0.812500
Epoch  4, Batch 394 - Loss:   482.

Epoch  5, Batch  70 - Loss:   710.0559 Validation Accuracy: 0.816406
Epoch  5, Batch  71 - Loss:   613.4961 Validation Accuracy: 0.812500
Epoch  5, Batch  72 - Loss:   526.4881 Validation Accuracy: 0.812500
Epoch  5, Batch  73 - Loss:   412.2386 Validation Accuracy: 0.820312
Epoch  5, Batch  74 - Loss:   501.3546 Validation Accuracy: 0.820312
Epoch  5, Batch  75 - Loss:   658.0072 Validation Accuracy: 0.812500
Epoch  5, Batch  76 - Loss:   523.9645 Validation Accuracy: 0.812500
Epoch  5, Batch  77 - Loss:   771.8018 Validation Accuracy: 0.816406
Epoch  5, Batch  78 - Loss:   391.3617 Validation Accuracy: 0.812500
Epoch  5, Batch  79 - Loss:   573.0260 Validation Accuracy: 0.808594
Epoch  5, Batch  80 - Loss:   401.6980 Validation Accuracy: 0.812500
Epoch  5, Batch  81 - Loss:   350.6841 Validation Accuracy: 0.808594
Epoch  5, Batch  82 - Loss:   914.1476 Validation Accuracy: 0.816406
Epoch  5, Batch  83 - Loss:   413.0289 Validation Accuracy: 0.816406
Epoch  5, Batch  84 - Loss:   573.

Epoch  5, Batch 189 - Loss:   533.8879 Validation Accuracy: 0.816406
Epoch  5, Batch 190 - Loss:   532.3406 Validation Accuracy: 0.816406
Epoch  5, Batch 191 - Loss:   475.0599 Validation Accuracy: 0.816406
Epoch  5, Batch 192 - Loss:   501.1294 Validation Accuracy: 0.812500
Epoch  5, Batch 193 - Loss:   570.7573 Validation Accuracy: 0.816406
Epoch  5, Batch 194 - Loss:   681.5639 Validation Accuracy: 0.816406
Epoch  5, Batch 195 - Loss:   389.6301 Validation Accuracy: 0.812500
Epoch  5, Batch 196 - Loss:   293.2315 Validation Accuracy: 0.812500
Epoch  5, Batch 197 - Loss:   495.0103 Validation Accuracy: 0.824219
Epoch  5, Batch 198 - Loss:   566.6413 Validation Accuracy: 0.824219
Epoch  5, Batch 199 - Loss:   568.3455 Validation Accuracy: 0.816406
Epoch  5, Batch 200 - Loss:   463.3155 Validation Accuracy: 0.820312
Epoch  5, Batch 201 - Loss:   688.6033 Validation Accuracy: 0.820312
Epoch  5, Batch 202 - Loss:   438.8413 Validation Accuracy: 0.828125
Epoch  5, Batch 203 - Loss:   744.

Epoch  5, Batch 308 - Loss:   655.8431 Validation Accuracy: 0.820312
Epoch  5, Batch 309 - Loss:   379.4587 Validation Accuracy: 0.820312
Epoch  5, Batch 310 - Loss:   497.1659 Validation Accuracy: 0.820312
Epoch  5, Batch 311 - Loss:   333.2354 Validation Accuracy: 0.820312
Epoch  5, Batch 312 - Loss:   606.7843 Validation Accuracy: 0.816406
Epoch  5, Batch 313 - Loss:   388.0068 Validation Accuracy: 0.820312
Epoch  5, Batch 314 - Loss:   764.3613 Validation Accuracy: 0.812500
Epoch  5, Batch 315 - Loss:   636.5018 Validation Accuracy: 0.812500
Epoch  5, Batch 316 - Loss:   522.8778 Validation Accuracy: 0.816406
Epoch  5, Batch 317 - Loss:   622.9617 Validation Accuracy: 0.816406
Epoch  5, Batch 318 - Loss:   447.9709 Validation Accuracy: 0.816406
Epoch  5, Batch 319 - Loss:   500.0457 Validation Accuracy: 0.816406
Epoch  5, Batch 320 - Loss:   436.5882 Validation Accuracy: 0.824219
Epoch  5, Batch 321 - Loss:   636.3926 Validation Accuracy: 0.816406
Epoch  5, Batch 322 - Loss:   466.

KeyboardInterrupt: 